In [ ]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.api as sm
import os

import petab
import pypesto
import pypesto.petab
from pypesto.optimize import minimize
from pypesto.engine import MultiProcessEngine
from pypesto.optimize.optimizer import FidesOptimizer
from pypesto.visualize import waterfall, parameters, profiles
from pypesto.visualize.model_fit import _get_simulation_rdatas
import pypesto.profile as profile
from pypesto.optimize import ScipyOptimizer
from pypesto.profile.options import ProfileOptions

Some plotting helper functions

In [ ]:
def hex_to_rgba_gradient(color1, color2, n):
    '''
    Create a gradient in rgba between two hex colors
    '''
    # Convert to rgba
    c1 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color1))
    c2 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color2))

    return [[(c1[i]*(n-j-1) + c2[i]*j)/(n-1) for i in range(4)] for j in range(n)]

# find the index for cut off based on Chi square distribution CI 95%
def find_cut_off_index(result, ci = 0.95):
    '''
    Find the cut off index for the data based on the Chi square distribution
    '''

    # calculate the chi square distribution
    cut_off_value = scipy.stats.chi2.ppf(ci, 1)

    # find the index
    best_fval = result.optimize_result.list[0].fval

    for i in range(len(result.optimize_result.list)):
        if result.optimize_result.list[i].fval > best_fval + cut_off_value:
            break
    
    return i - 1

In [ ]:
# Plot setting
plt.rcParams['font.size'] = 30
plt.rcParams['font.family'] = 'Arial'

dpi = 100
wid = int(2560/dpi)
hei = int(1600/dpi)

red_color = '#f78884'
blue_color = '#06688c'

# Define the folder where you want to save the figures
folder_path = "../../figure/baseline_model/"
# If the folder does not exist, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# import the petab problem and load the optimization result

In [ ]:
# number of optimization runs
n_runs, max_iter = 5000, 5000
# set the random seed
np.random.seed(500)

In [ ]:
# import the petab problem
petab_yaml = 'petab_files/baseline_model.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=False,
        model_name=f"Baseline_Model",
    ).create_problem(force_compile=True, verbose=False)
problem.objective.amici_model.setAllStatesNonNegative()

# some model properties
print("Model parameters:", list(problem.objective.amici_model.getParameterIds()), "\n")
print("Model const parameters:", list(problem.objective.amici_model.getFixedParameterIds()), "\n")
print("Model outputs:   ", list(problem.objective.amici_model.getObservableIds()), "\n")
print("Model states:    ", list(problem.objective.amici_model.getStateIds()), "\n")

In [ ]:
# load result history from file
result = pypesto.store.read_result('optimization_history/baseline_model.hdf5')

# print result summary
print(result.summary())

In [ ]:
parameters_from_result = dict(zip(problem.x_names, result.optimize_result.list[0]['x']))
# Scale all parameters and put them into a dictionary
scaled_parameters = {key: 10**value for key, value in parameters_from_result.items()}

# Print the scaled parameters
print("Scaled parameters:")
for key, value in scaled_parameters.items():
    print(f"{key}: {value}")

# Obtain data and visualize the fitting result

In [ ]:
return_dict = problem.objective(result.optimize_result.list[0].x, return_dict=True)
rdatas = return_dict['rdatas']
edatas = problem.objective.edatas
x_axis = [edata.id for edata in edatas]
simulation = [rdata.y.reshape(5, -1)[:,0] for rdata in rdatas]
data = [np.array(edata.getObservedData()).reshape(5, -1) for edata in edatas]

In [ ]:
# get the statistics for 95% CI
cut_off_index = find_cut_off_index(result)
# define as in petab_files_creation.ipynb to convert the data back to tumor volume
s = 3510.7678534742176

In [ ]:
"""
visualize the temporal dynamics of the virus, uninfected and infected tumor cells using the fitted model from the result
from day 3 to day 7
get the simulation results for the optimized parameters
"""

amici_model = problem.objective.amici_model

species_to_plot = ['U', 'I', 'V']

# simulate from day 3 to day 12
stop_day = 4
timepoints = np.linspace(start=0, stop=stop_day, num=100)

simulation_rdatas = _get_simulation_rdatas(
    result=result,
    problem=problem,
    start_index = 0,
    simulation_timepoints=timepoints,
)

In [ ]:
# Create a figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(20, 8))

# Panel a
for i in range(10):
    axs[0].plot(np.array([3,4,5,6,7]), data[0][:, i]*s, 
             marker='o', linestyle='-', color=red_color, alpha=0.3, label='ctrl (data)' if i == 0 else "")
axs[0].plot(timepoints+3, simulation_rdatas[0]['y']*s, 
         linestyle='-', lw= 3, color=red_color, alpha=1)

for i in range(10):
    axs[0].plot(np.array([3,4,5,6,7]), data[1][:, i]*s, 
             marker='o', linestyle='-', color=blue_color, alpha=0.3, label='vvDD (data)' if i == 0 else "")
axs[0].plot(timepoints+3, simulation_rdatas[1]['y']*s, 
         linestyle='-', lw= 3, color=blue_color, alpha=1)

axs[0].set_xlabel('Time (days)')
axs[0].set_ylabel(r'Tumor Volume ($\mu m^3$)')
axs[0].set_title('Model vs data (individual)', pad=20, loc='center')
axs[0].grid(False)
axs[0].set_xticks([3, 4, 5, 6, 7])
axs[0].text(-0.125, 1.22, 'a', transform=axs[0].transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

# Remove top and right lines
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

# Make spines thicker
axs[0].spines['left'].set_linewidth(2)
axs[0].spines['bottom'].set_linewidth(2)

# Make ticks thicker
axs[0].tick_params(width=2)

# Mark lines directly
axs[0].text(7, simulation_rdatas[0]['y'][-1]*s, '  ctrl (model)', color=red_color, fontsize=20)
axs[0].text(7, simulation_rdatas[1]['y'][-1]*s, '  vvDD (model)', color=blue_color, fontsize=20)

# Panel b
# Calculate mean and standard error for vvDD
vvDD_mean = (data[1]*s).mean(axis=1)
vvDD_se = (data[1]*s).std(axis=1)
# Calculate mean and standard error for PBS
pbs_mean = (data[0]*s).mean(axis=1)
pbs_se = (data[0]*s).std(axis=1)

# Plot provided data
axs[1].errorbar(np.array([3,4,5,6,7]), vvDD_mean, 
             yerr=[vvDD_se],
             fmt='o--', color=blue_color, alpha=0.3, ecolor='lightblue', capsize=5, label='vvDD (data)')
axs[1].errorbar(np.array([3,4,5,6,7]), pbs_mean, 
             yerr=[pbs_se],
             fmt='o--', color=red_color, alpha=0.3, ecolor='lightcoral', capsize=5, label='ctrl (data)')
# Plot simulated data
axs[1].plot(timepoints+3, simulation_rdatas[0]['y']*s, 
         linestyle='-', lw= 3, color=red_color, alpha=1)
axs[1].plot(timepoints+3, simulation_rdatas[1]['y']*s, 
         linestyle='-', lw= 3, color=blue_color, alpha=1)

axs[1].set_xlabel('Time (days)')
axs[1].set_ylabel(r'Tumor Volume ($\mu m^3$)') 
axs[1].set_title('Model vs data (Population)', pad=20, loc='center')
axs[1].grid(False)
axs[1].set_xticks([3, 4, 5, 6, 7])
axs[1].text(-0.2, 1.22, 'b', transform=axs[1].transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

# Remove top and right lines
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

# Make spines thicker
axs[1].spines['left'].set_linewidth(2)
axs[1].spines['bottom'].set_linewidth(2)

# Make ticks thicker
axs[1].tick_params(width=2)

# Mark lines directly
axs[1].text(7, simulation_rdatas[0]['y'][-1]*s, '  ctrl (model)', color=red_color, fontsize=20)
axs[1].text(7, simulation_rdatas[1]['y'][-1]*s, '  vvDD (model)', color=blue_color, fontsize=20)

# Adjust layout and save the figure
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=2, frameon=False, bbox_to_anchor=(0.5, -0.05))
plt.tight_layout()
plt.subplots_adjust(wspace=0.75)  # Add more space between subplots
plt.savefig(folder_path + 'combined_fit.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:

# Create a 3x2 subplot layout with shared y-axis for each row
fig, axs = plt.subplots(3, 2, figsize=(20, 10), sharex=True, sharey='row')

panel_labels = ['a', 'b']

space = -0.145
for i, species in enumerate(species_to_plot):
    for c_, condition in enumerate(['ctrl', 'vvDD']):
        ax = axs[i, c_]
        color = red_color if condition == 'ctrl' else blue_color
        ax.plot(timepoints, simulation_rdatas[c_]['x'][:, amici_model.getStateIds().index(species)], color=color, lw=4)
        
        ax.set_xticks(np.arange(0, stop_day + 1, 1))
        ax.set_xticklabels(np.arange(3, stop_day + 4, 1))
        if i == 0:
            ax.set_title(f"{condition}", pad=20, loc='center')
            ax.set_ylabel(r'[U]', labelpad=20)
        else:
            ax.set_title(f" ", pad=-10, loc='center')
        if i == 1:
            ax.set_ylabel(r'[I]', labelpad=20)
        if i == 2:
            ax.set_ylabel(r'[V]', labelpad=20)
            ax.set_xlabel('Time (days)')

        ax.yaxis.set_tick_params(labelleft=True)
        
        # Remove top and right lines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # Make spines thicker
        ax.spines['left'].set_linewidth(2)
        ax.spines['bottom'].set_linewidth(2)

        # Make ticks thicker
        ax.tick_params(width=2)
        if i == 0:
            # Add panel labels
            ax.text(space-0.025, 1.25, panel_labels[i * 2 + c_], transform=ax.transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

# Adjust the position of the y-axis labels
for ax in axs[:, 0]:
    ax.yaxis.set_label_coords(space, 0.5)
# Adjust the position of the y-axis labels
for ax in axs[:, 1]:
    ax.yaxis.set_label_coords(space, 0.5)

plt.subplots_adjust(wspace=0.3, hspace=0.8)  # Adjust space between subplots
# plt.savefig(folder_path + 'state_trajectories.pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# calculate the 1-sigma for the simulation
sigma_a = scaled_parameters['sigma_a']
sigma_b = scaled_parameters['sigma_b']

sigma_pbs = np.sqrt(sigma_a**2 + (simulation_rdatas[0]['y'].reshape(1, -1)[0] * sigma_b)**2)
sigma_vvDD = np.sqrt(sigma_a**2 + (simulation_rdatas[1]['y'].reshape(1, -1)[0] * sigma_b)**2)

In [ ]:
# Create a 3x2 subplot layout with shared x-axis for all subplots
"""
the first column is for the model vs data population plot
the second column is for the state trajectories plot
"""
fig = plt.figure(figsize=(12, 11))

gs = fig.add_gridspec(3, 2)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[2, 1])

# Panel a: Model vs data (population)
ax1.errorbar(np.array([3, 4, 5, 6, 7]), pbs_mean, yerr=[pbs_se], fmt='o--', lw=5, color=red_color, alpha=0.4, ecolor=red_color, capsize=5, capthick=5, label='control (data)', zorder=1)
ax1.errorbar(np.array([3, 4, 5, 6, 7]), vvDD_mean, yerr=[vvDD_se], fmt='o--', lw=5, color=blue_color, alpha=0.4, ecolor=blue_color, capsize=5, capthick=5, label='vvDD (data)', zorder=2)
ax1.plot(timepoints + 3, simulation_rdatas[0]['y'].reshape(1, -1)[0] * s, linestyle='-', lw=5, color=red_color, alpha=1, label='control (simulation)', zorder=3)
ax1.plot(timepoints + 3, simulation_rdatas[1]['y'].reshape(1, -1)[0] * s, linestyle='-', lw=5, color=blue_color, alpha=1, label='vvDD (simulation)', zorder=4)
ax1.fill_between(timepoints + 3, simulation_rdatas[0]['y'].reshape(1, -1)[0] * s - s * sigma_pbs, simulation_rdatas[0]['y'].reshape(1, -1)[0] * s + s * sigma_pbs, color=red_color, alpha=0.2, edgecolor='none', zorder=3)
ax1.fill_between(timepoints + 3, simulation_rdatas[1]['y'].reshape(1, -1)[0] * s - s * sigma_vvDD, simulation_rdatas[1]['y'].reshape(1, -1)[0] * s + s * sigma_vvDD, color=blue_color, alpha=0.2, edgecolor='none', zorder=4)
ax1.set_xlabel('Time [days]')
ax1.set_ylabel(r'Tumor Volume [$\mu m^3$]')
# ax1.set_title('Model vs data (population)', pad=20, loc='center')
ax1.grid(False)
ax1.set_xticks([3, 4, 5, 6, 7])

ax1.yaxis.set_tick_params(labelleft=True)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['left'].set_linewidth(2)
ax1.spines['bottom'].set_linewidth(2)
ax1.tick_params(width=2)

# Panel b: State trajectories
axs = [ax2, ax3, ax4]
for i, species in enumerate(species_to_plot):
    for c_, condition in enumerate(['ctrl', 'vvDD']):
        ax = axs[i]
        color = red_color if condition == 'ctrl' else blue_color
        ax.plot(timepoints + 3, simulation_rdatas[c_]['x'][:, amici_model.getStateIds().index(species)], color=color, lw=5)
        ax.set_xticks(np.arange(3, stop_day + 4, 1))
        ax.set_xticklabels(np.arange(3, stop_day + 4, 1))
        if i == 0:
            # ax.set_title(f"{condition}", pad=20, loc='center')
            ax.set_ylabel(r'[$U$]', labelpad=20)
        elif i == 1:
            ax.set_ylabel(r'[$I$]', labelpad=20)
        elif i == 2:
            ax.set_ylabel(r'[$V$]', labelpad=20)
            ax.set_xlabel('Time [days]')
        ax.yaxis.set_tick_params(labelleft=True)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_linewidth(2)
        ax.spines['bottom'].set_linewidth(2)
        ax.tick_params(width=2)

# Adjust the position of the y-axis labels
for ax in axs:
    ax.yaxis.set_label_coords(-0.35, 0.5)

ax1.text(-0.2, 1.1, 'b', transform=ax1.transAxes, fontsize=40, fontweight='bold', va='top', ha='right')
ax2.text(-0.4, 1.5, 'c', transform=ax2.transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=2, frameon=False, bbox_to_anchor=(0.5, -0.125))

plt.subplots_adjust(wspace=0.3, hspace=0.1)  # Adjust space between subplots

plt.tight_layout()
plt.savefig(folder_path + 'pop_and_states.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Parameter estimation analysis

In [ ]:
# correction of the code error
result.problem.x_scales = ['log10'] * len(result.problem.x_names)

In [ ]:
plt.rcParams.update({'font.size': 30})

fig = plt.figure(figsize=(wid, hei))

gs = fig.add_gridspec(2, 3)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 2])

waterfall(result, ax=ax1)
ax1.set_ylabel('Objective value') 

# Parameter plots in the second row
pypesto.visualize.parameters(result, ax=ax2, plot_inner_parameters=False, start_indices=2000, colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', 2000))
pypesto.visualize.parameters(result, ax=ax3, plot_inner_parameters=False, start_indices=500, colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', 500))
pypesto.visualize.parameters(result, ax=ax4, plot_inner_parameters=False, start_indices=100, colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', 100))

ax2.set_title('Top 2000', fontsize=30)
ax3.set_title('Top 500', fontsize=30)
ax4.set_title('Top 100', fontsize=30)

# Set all the x-axis, x and y labels to have fontsize 30
for ax in [ax2, ax3, ax4]:
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=20, rotation=45)
    ax.set_xlabel('Parameter Value', fontsize=30)
ax2.set_ylabel('Parameter', fontsize=30)

# Remove top and right lines and make lines and ticks thicker
for i, ax in enumerate([ax1, ax2, ax3, ax4]):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.tick_params(width=2)
    # Add panel labels
    if i == 0:
        ax.text(-0.115, 1.1, 'a', transform=ax.transAxes, fontsize=40, fontweight='bold', va='top', ha='right')
    else:
        ax.text(-0.55, 1.1, (['b', 'c', 'd'])[i-1], transform=ax.transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

plt.tight_layout()
plt.savefig(os.path.join(folder_path, 'waterfall_parameters_plot.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

# Profiling

In [ ]:
result = profile.parameter_profile(
    problem=problem,
    result=result,
    engine=MultiProcessEngine(),
    optimizer=ScipyOptimizer(),
    result_index=0,
    profile_options=ProfileOptions(whole_path=False),
)

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(14, 8))
ax = ax.flatten()  # Flatten the 2D array to 1D for easier iteration
labels = ["$\\rho$", "$\\kappa$", "$\\psi$", "$\\beta$", "$\\alpha$", "$\\delta$", "$\\sigma_{a}$", "$\\sigma_{b}$"]

profiles(result, show_bounds=True, size=(12, 10), colors=[0,0,0,1], profile_list_ids=len(result.profile_result.list)-1, quality_colors=False, ax=ax)

for i, a in enumerate(ax):
    a.set_ylim([0, 1.1])
    a.spines['top'].set_visible(False)
    a.spines['right'].set_visible(False)
    a.spines['left'].set_linewidth(2)
    a.spines['bottom'].set_linewidth(2)
    a.tick_params(width=2, labelsize=25)  # Set tick label size to 20
    for label in a.get_xticklabels():
        label.set_rotation(45)
    a.set_xlabel(rf'{labels[i]}', fontsize=40)  # Change x-axis label to math notation and increase font size
    a.set_ylabel('')  # Remove innate y-label
    if len(a.get_xticks()) == 12:
        a.set_xticks(a.get_xticks()[::3])
    if len(a.get_xticks()) == 9 or len(a.get_xticks()) == 8 or len(a.get_xticks()) == 7:
        a.set_xticks(a.get_xticks()[::2])
    if i == 0:
        a.set_xticklabels([f'${{{10**(tick):.2f}}}$' for tick in a.get_xticks()])
    else:
        a.set_xticklabels([f'$10^{{{int(tick)}}}$' for tick in a.get_xticks()])

# Add a common y-label
fig.text(0.03, 0.5, 'Likelihood Ratio', va='center', rotation='vertical', fontsize=30)
# Adjust the position of the y-axis labels
for a in ax:
    a.xaxis.set_label_coords(0.55, -0.45)

plt.subplots_adjust(wspace=0.5, hspace=1)
plt.savefig(os.path.join(folder_path, 'profile_plot_res.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

# AIC and residue plot

In [ ]:
n_measurements = 100
n_parameters = len(result.optimize_result.list[0]['x'])

# Calculate AIC
AIC = 2 * n_parameters + 2 * result.optimize_result.list[0]['fval']
print(f"AIC: {AIC}")

# Calculate AIC for the small dataset
AIC_small = AIC + (2. * n_parameters**2 + 2.*n_parameters)/(n_measurements - n_parameters - 1)
print(f"AICc: {AIC_small}")